In [1]:
import pandas as pd

import numpy as np
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import _tree
import random

from skopt.space import Real, Categorical, Integer
random.seed(42)

In [2]:
def _sample_search_space(param_space, categorical_columns, num_samples = 1000):
    """
        Uniformly sample the search space for generating explanation dataset. Maintains the original order of search space.
        Sensitive to order: place categoricals first and everything else later.

        Args:
        - param_space (param space): parameter space
        - categorical_columns (list): list of categorical variables
        - num_samples (int): number of generated samples

        Returns:
        - new_data (numpy array): the augmented dataset
        """
    samples = []
    
    for _ in range(num_samples):
        sample = [param.rvs() for param in param_space]
        samples.append(sample)
    
    # Create a DataFrame with the samples
    param_names = [param.name for param in param_space]
    df = pd.DataFrame(samples, columns=param_names)
    
    # One-hot encode categorical variables and add dynamic prefixes
    encoded_columns = []
    for col in categorical_columns:
        prefix = f'categorical_{col}'
        encoded = pd.get_dummies(df[col], prefix=prefix).astype(float)
        encoded_columns.extend(encoded.columns)
        df = pd.concat([df.drop(col, axis=1), encoded], axis=1)
    
    # Rearrange columns: categorical columns in front, followed by non-categorical columns
    ordered_columns = []
    for col in param_space:
        if col.name in categorical_columns:
            ordered_columns.extend([col_name for col_name in df.columns if col_name.startswith(f'categorical_{col.name}')])
        elif col.name in df.columns:
            ordered_columns.append(col.name)
    
    # Now you have the DataFrame with categorical variables in the front and non-categorical variables in their original order
    df = df[ordered_columns]
    
    # Convert numerical variables from lists to floats
    for col in param_names:
        if col not in categorical_columns:
            df[col] = df[col]
    df = df.to_numpy().astype(np.float32)
    return df

In [3]:
df = pd.read_csv('mnist_xception_25.csv')
df = df.drop(columns=['training_epoch_accuracy','training_epoch_loss','validation_epoch_loss','validaiton_evaluation_accuracy_vs_iterations','validation_evaluation_loss_vs_iterations'])

In [4]:
ye = df['validation_epoch_accuracy']
y = ye.to_numpy()

Xe = df.drop(columns='validation_epoch_accuracy')
dum1 = pd.get_dummies(Xe['conv2d_num_filters'],prefix = 'categorical_conv2dnumfilters')
dum2 = pd.get_dummies(Xe['activation'],prefix = 'categorical_activation')
dum3 = pd.get_dummies(Xe['pooling'],prefix = 'categorical_pooling')
dum4 = pd.get_dummies(Xe['kernel_size'],prefix = 'categorical_kernelsize')
dum5 = pd.get_dummies(Xe['dense_use_bn'],prefix = 'categorical_denseusebn')
Xe = Xe.drop(columns = ['conv2d_num_filters','activation','pooling','kernel_size','dense_use_bn'])
Xe = pd.concat([dum1, dum2, dum3, dum4, dum5, Xe], axis=1)

param_space = [
    Categorical(categories=['64','32', '128'], name='conv2d_num_filters'),
    Categorical(categories=['relu', 'selu'], name='activation'),
    Categorical(categories=['flatten', 'avg', 'max'], name='pooling'),
    Categorical(categories=['5','3'], name='kernel_size'),
    Categorical(categories=['1', '0'], name='dense_use_bn'),
    Integer(low=128, high=768, name='sep_num_filters'),
    Integer(low=0, high=1, name='dropout_rate'),
    Integer(low=1, high=5, name='num_dense_layers'),
    Integer(low=1, high=8, name='num_residual_blocks'),
    Integer(low=0, high=1, name='learning_rate'),
    Integer(low=2, high=4, name='initial_strides'),
]

# Define the categorical columns
categorical_columns = ['conv2d_num_filters','activation','pooling','kernel_size','dense_use_bn']


print (Xe.iloc[np.argmin(ye.values)])
print (np.max(ye.values))

categorical_conv2dnumfilters_32       0.000
categorical_conv2dnumfilters_64       1.000
categorical_conv2dnumfilters_128      0.000
categorical_activation_relu           0.000
categorical_activation_selu           1.000
categorical_pooling_avg               0.000
categorical_pooling_flatten           1.000
categorical_pooling_max               0.000
categorical_kernelsize_3              1.000
categorical_kernelsize_5              0.000
categorical_denseusebn_0              1.000
categorical_denseusebn_1              0.000
sep_num_filters                     640.000
initial_strides                       2.000
learning_rate                         0.001
num_dense_layers                      2.000
num_residual_blocks                   7.000
dropout_rate                          0.300
Name: 2, dtype: float64
0.973733306


In [5]:
import GPy

def _train_gp(x_train, y_train):
    """
        Train a GPR model from Gpy library.

        Args:
        - X_train (numpy array): input data
        - Y_train (numpy array): ground truth data
        - n_samples (int): number of times to sample the posterior of the trained gp
        - sample_points (numpy array): new input X locations where the posterior would be sampled

        Returns:
        - model (trained gp model): trained gp model
        - mean_y (numpy array): mean of the Y from the sampled posterior
        - std_y (numpy array): std dev of the Y from the sampled posterior
        - sum_log_likelihoods (numpy array): returns the likelihood of observing
          each input point given the trained model
    """
    kernel = GPy.kern.RBF(input_dim=x_train.shape[1])
    model = GPy.models.GPRegression(x_train, y_train, kernel)
    model.optimize()
    return model

def augment_with_noise(data, n_samples=1000):
    """
    Add noise to the input data to create new points and generate a new dataset
    with the desired number of samples.

    Args:
    - data (numpy array): the input data
    - num_samples (int): the number of samples to generate
    - noise_factor (float): the magnitude of the noise to add to the data (default: 0.01)

    Returns:
    - new_data (numpy array): the augmented dataset
    """
    n_points, n_dims = data.shape
    new_data = np.empty((n_samples, n_dims))  # Create a new array for the augmented data

    for i in range(n_samples):
        noise = np.random.uniform(low=-0.01, high=0.01, size=n_dims)
        new_data[i] = data[np.random.randint(n_points)] + noise

    return new_data
    
model = _train_gp(Xe.to_numpy(), y.reshape(-1, 1))
test_set = _sample_search_space(param_space, categorical_columns, num_samples = 100)
test_set = augment_with_noise(test_set)
mean_y, var_y = model.predict_noiseless(test_set, full_cov=False)
flattened_mean = [item for sublist in mean_y for item in sublist]
flattened_var = [item for sublist in var_y for item in sublist]
std = np.sqrt(flattened_var)

samp = _sample_search_space(param_space, categorical_columns, num_samples = 1000)
samp = augment_with_noise(samp)
yee, var = model.predict_noiseless(samp, full_cov=False)
yee = np.squeeze(yee, axis=1)

In [8]:
from rulekit import RuleKit
from rulekit.regression import RuleRegressor
from rulekit.params import Measures
RuleKit.init()

reg = RuleRegressor(
    induction_measure=Measures.C2,
    pruning_measure=Measures.C2,
    voting_measure=Measures.C2,
)
reg.fit(samp, yee)
predictions = reg.predict(test_set)
ru = []
for rule in reg.model.rules:
    ru.append(str(rule))
    print (rule)

print(len(ru))

from itertools import combinations

# Given rule list in the new representation
rule_list = ru

# Function to extract features from a rule
def extract_features(rule):
    # Extract only the part before "THEN" and split by "AND"
    features = set(rule.split('IF')[1].split(" THEN")[0].split(" AND"))
    return features

# Function to calculate Jaccard similarity between two rules
def jaccard_similarity(rule1, rule2):
    features1 = extract_features(rule1)
  
    features2 = extract_features(rule2)
   
    intersection = len(features1.intersection(features2))
    union = len(features1.union(features2))
    
    if union == 0:
        return 0.0  # Handle the case where both sets are empty
    else:
        return intersection / union

# Calculate Jaccard similarity between all pairs of rules and collect them
similarities = []
for rule1, rule2 in combinations(rule_list, 2):
    similarity = jaccard_similarity(rule1, rule2)
    similarities.append(similarity)

# Calculate the mean Jaccard similarity for the entire rule set
mean_similarity = sum(similarities) / len(similarities)

print(f"Mean Jaccard Similarity for the Rule Set: {mean_similarity:.2f}")

## Define your bounds
lower_bounds = [flattened_mean[i] - 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]
upper_bounds = [flattened_mean[i] + 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]


# Initialize a count for predictions within bounds
count_within_bounds = sum(1 for prediction, lower_bound, upper_bound in zip(predictions, lower_bounds, upper_bounds) if lower_bound <= prediction <= upper_bound)

# Calculate the percentage within bounds
percentage_within_bounds = (count_within_bounds / len(predictions)) 

# Print the result
print(f"Percentage of predictions within bounds: {percentage_within_bounds}%")

IF att13 = (-inf, 132.00) THEN label = {1.00} [1.00,1.00]
IF att5 = (-inf, 1.01) AND att1 = (-inf, 1.00) AND att13 = <759.50, inf) THEN label = {1.00} [1.00,1.00]
IF att13 = <757.50, inf) THEN label = {1.00} [1.00,1.00]
IF att6 = (-inf, 0.50) AND att2 = <-0.0068, inf) AND att13 = <750.01, 766.50) AND att3 = <-0.0077, inf) THEN label = {1.00} [1.00,1.00]
IF att5 = (-inf, 1.01) AND att13 = <738.49, 744.49) THEN label = {1.00} [1.00,1.00]
IF att1 = (-inf, 1.00) AND att13 = <734.00, 738.49) AND att4 = (-inf, 0.50) THEN label = {1.00} [1.00,1.00]
IF att9 = <-0.0027, inf) AND att6 = (-inf, 1.00) AND att5 = <-0.0077, inf) AND att17 = (-inf, 1.00) AND att7 = (-inf, 1.01) AND att13 = <734.00, 745.50) AND att3 = (-inf, 1.00) AND att12 = <-0.0059, inf) THEN label = {1.00} [1.00,1.00]
IF att17 = <-2.0E-5, inf) AND att8 = <-0.0066, 0.50) AND att11 = (-inf, 1.00) AND att13 = <734.00, inf) AND att3 = (-inf, 1.00) THEN label = {1.00} [1.00,1.00]
IF att9 = <-0.0056, inf) AND att5 = (-inf, 1.01) AND att

In [6]:
clf = DecisionTreeRegressor(random_state=0)
clf.fit(samp, yee)
pred = clf.predict(test_set)


# Define your bounds
lower_bounds = [flattened_mean[i] - 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]
upper_bounds = [flattened_mean[i] + 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]

# Initialize a count for predictions within bounds
pr_count_within_bounds = sum(1 for prediction, lower_bound, upper_bound in zip(pred, lower_bounds, upper_bounds) if lower_bound <= prediction <= upper_bound)

# Calculate the percentage within bounds
percentage_within_bounds = (pr_count_within_bounds / len(pred)) 

# Print the result
print(f"Percentage of predictions within bounds: {percentage_within_bounds}%")

def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        #rule += " then "
        if class_names is None:
            s = 0
        #    rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        #rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules
rules = get_rules(clf, Xe.columns, None)
for r in rules:
    print(r)
len(rules)

Percentage of predictions within bounds: 1.0%
if (sep_num_filters > 430.501) and (sep_num_filters <= 587.498) and (sep_num_filters <= 499.005) and (sep_num_filters > 458.498)
if (sep_num_filters > 430.501) and (sep_num_filters > 587.498) and (sep_num_filters <= 671.492) and (sep_num_filters > 629.507)
if (sep_num_filters > 430.501) and (sep_num_filters > 587.498) and (sep_num_filters > 671.492) and (sep_num_filters <= 718.007) and (sep_num_filters > 691.995)
if (sep_num_filters > 430.501) and (sep_num_filters <= 587.498) and (sep_num_filters > 499.005) and (sep_num_filters <= 543.497) and (sep_num_filters > 514.005)
if (sep_num_filters <= 430.501) and (sep_num_filters > 273.506) and (sep_num_filters > 359.499) and (sep_num_filters > 389.506) and (sep_num_filters > 408.998)
if (sep_num_filters > 430.501) and (sep_num_filters > 587.498) and (sep_num_filters > 671.492) and (sep_num_filters > 718.007) and (sep_num_filters > 741.005)
if (sep_num_filters <= 430.501) and (sep_num_filters <= 2

49

In [7]:
# from itertools import combinations

# # Given rule list
# rule_list = rules

# # Function to extract features from a rule
# def extract_features(rule):
#     features = set(rule.split("and"))
#     return features

# # Function to calculate Jaccard similarity between two rules
# def jaccard_similarity(rule1, rule2):
#     features1 = extract_features(rule1)
#     features2 = extract_features(rule2)
#     intersection = len(features1.intersection(features2))
#     union = len(features1.union(features2))
    
#     if union == 0:
#         return 0.0  # Handle the case where both sets are empty
#     else:
#         return intersection / union

# # Calculate Jaccard similarity between all pairs of rules and collect them
# similarities = []
# for rule1, rule2 in combinations(rule_list, 2):
#     similarity = jaccard_similarity(rule1, rule2)
#     similarities.append(similarity)

# # Calculate the mean Jaccard similarity for the entire rule set
# mean_similarity = sum(similarities) / len(similarities)

# print(f"Mean Jaccard Similarity for the Rule Set: {mean_similarity:.2f}")

In [7]:
from skrules import SkopeRules
from sklearn.preprocessing import KBinsDiscretizer
yee = yee/np.max(yee)

# transform the dataset with KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=2, encode="ordinal", strategy='kmeans')
Y_binned = enc.fit_transform(yee.reshape(-1, 1))

clf1 = SkopeRules(
    max_depth_duplication=3, max_depth=3, max_features=0.5,
    max_samples_features=0.5, random_state=0, n_estimators=20,
    feature_names=Xe.columns, recall_min=0.04, precision_min=0.6)

clf1.fit(samp, Y_binned)

p = clf.predict(test_set)


# Define your bounds
lower_bounds = [flattened_mean[i] - 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]
upper_bounds = [flattened_mean[i] + 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]

# Initialize a count for predictions within bounds
clf_count_within_bounds = sum(1 for prediction, lower_bound, upper_bound in zip(p, lower_bounds, upper_bounds) if lower_bound <= prediction <= upper_bound)

# Calculate the percentage within bounds
percentage_within_bounds = (clf_count_within_bounds / len(p)) 

# Print the result
print(f"Percentage of predictions within bounds: {percentage_within_bounds}%")

print(len(clf1.rules_))
print(clf1.rules_)

 c:\users\uif75022\phddevstuff\rxboplusdev\rxboplus\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning:A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 c:\users\uif75022\phddevstuff\rxboplusdev\rxboplus\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning:`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
 c:\users\uif75022\phddevstuff\rxboplusdev\rxboplus\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning:`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.


Percentage of predictions within bounds: 1.0%
13
[('sep_num_filters > 435.5010986328125', (1.0, 1.0, 2)), ('sep_num_filters > 435.5010986328125 and num_dense_layers > 0.999290943145752', (1.0, 0.9280575539568345, 2)), ('sep_num_filters > 435.5010986328125 and learning_rate > 1.00328928232193', (1.0, 0.8157248157248157, 2)), ('sep_num_filters > 433.502685546875 and num_dense_layers <= 2.9913665056228638', (1.0, 0.28368794326241137, 2)), ('initial_strides > -0.00903868768364191 and num_residual_blocks > -0.003957652719691396 and dropout_rate > 3.9937790632247925', (0.6574585635359116, 0.2853717026378897, 2)), ('sep_num_filters > 426.0013122558594 and learning_rate <= 1.0070645213127136', (1.0, 0.23300970873786409, 2)), ('categorical_denseusebn_1 <= 1.008371889591217 and dropout_rate > 3.991756796836853 and categorical_activation_relu <= 0.9945928752422333', (0.6666666666666666, 0.21247113163972287, 2)), ('categorical_conv2dnumfilters_128 <= 0.9902361929416656 and categorical_activation_s